In [87]:
nct_ids = ["NCT04457596", "NCT05774951", "NCT04266249", "NCT05514054"]

In [88]:
import re
from datetime import timedelta

from requests_cache import CachedSession
from tqdm.notebook import tqdm

session = CachedSession(expire_after=timedelta(days=14))

studies = []
for id in tqdm(nct_ids):
    res = session.get(f"https://clinicaltrials.gov/api/v2/studies/{id}")
    res.raise_for_status()
    data = res.text
    studies.append((id, data))

  0%|          | 0/4 [00:00<?, ?it/s]

In [89]:
from functools import cmp_to_key
import json

keywordPattern = r"(breast)"


def sort_if_contains(a: tuple[str, str], b: tuple[str, str]):
    if re.findall(keywordPattern, a[1], re.I):
        return -1
    if re.findall(keywordPattern, b[1], re.I):
        return 1
    return 0


studies = sorted(studies, key=cmp_to_key(sort_if_contains))
matches = 0
delimiter = False
for id, study in studies:
    if found := re.findall(keywordPattern, study, re.I):
      matches += 1
      print(f"{id}:", found)
    else:
      if not delimiter:
         print('------------------------------------------')
         delimiter = True
      conditions = json.loads(study)["protocolSection"]["conditionsModule"]["conditions"]
      print(f"{id}:", conditions)
print(matches/len(studies))

NCT05514054: ['Breast', 'Breast', 'breast', 'Breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast']
NCT04266249: ['Breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast', 'Breast']
NCT05774951: ['Breast', 'Breast', 'breast', 'breast', 'breast', 'Breast', 'Breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'breast', 'Breast'

In [90]:
matches, len(studies)

(4, 4)

In [ ]:
import json

print(
    json.dumps(
        json.loads(
            session.cache.get_response(key="8ec542ba8e7e0027").content.decode("utf-8")
        ),
        indent=1,
    )
)
# for response in session.cache.filter():
#     print(response.cache_key if 'NCT03994796' in response.url else None)